Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE".

---

## Blatt 09

- **Jede(r)** Studierende muss die Aufgaben einreichen.
Wenn Sie die Aufgaben nicht selbst einreichen, erhalten Sie 0 Punkte für die Aufgabe.
- Sie werden ermutigt, in Gruppen von bis zu vier Personen an den Lösungen zu arbeiten.
    - Bei Einreichungen solcher Einreichungsgruppen speichern die Gruppen die imt-Logins aller Mitglieder in der Variable `AUTHORS`, z.B: `AUTHORS="first-imtlogin second-imtlogin"`.
    - Jedes Mitglied der Gruppen **muss das Assignment einreichen**.
    - Verwenden Sie dazu die vorbereitete Zelle:

**Anmerkung:** Löschen Sie *nicht* die Befehlshistorie (z.B. durch Ausführen von `history -c`) in *Ihrem* Code!

In [ ]:
AUTHORS=""

In [ ]:
[[ -d $SHEET_DIR ]] && cd $SHEET_DIR  # Ensure this cell can be executed repeatedly
rm -rf $SHEET_DIR/workdir-*  # Delete old work directories
SHEET_DIR=$(pwd)
WORK_DIR=$(mktemp -d -p $SHEET_DIR workdir-XXXXX)  # Create work directory
cd $WORK_DIR  # Change into work directory

## Aufgabe 1 - Buffers and Traps

Erstellen Sie ein Skript namens `buffer.sh` im Verzeichnis `$WORK_DIR` (das Arbeitsverzeichnis, das am Anfang dieses Notebooks definiert wurde).
Stellen Sie sicher, dass das Skript ausführbar ist.
Sie können es in eine Datei im `$SHEET_DIR` schreiben und von dort ins `$WORK_DIR` kopieren, es mit einem Heredoc erstellen, oder wie auch immer Sie wollen.
Sie müssen nur sicherstellen, dass das Skript im `$WORK_DIR` vorhanden, korrekt benannt und ausführbar ist.

Das Skript soll folgenden Anforderungen genügen:

- Lies (zeilenweise) Text aus der Standardeingabe `stdin` und speichere diesen in einer Art Buffer. Verwenden sie dabei auf **keinen Fall den Befehl mkfifo und named pipes(FIFO)**, sondern nutzen sie einen anderen Weg um Buffer zu schreiben.

- Leere den Puffer, wenn das Skript ein `USR1`-Signal (`SIGUSR1`) empfängt, d.h. (in dieser Reihenfolge):
    1. Printe die Zeile `flushing buffer` in der Standardfehlerausgabe `stderr`.
    2. Printe den gesamten Inhalt des Puffers in die Standardausgabe `stdout`.
    3. Lösche den Inhalt des Puffers.
    
- Wenn das Skript beendet wird (s. **[1]**), soll es sich wie folgt verhalten (in dieser Reihenfolge):
    1. Printe die Zeile `script shutting down` in der Standardfehlerausgabe `stderr`.
    2. Leere den Puffer ein letztes Mal (siehe obige Definition von "Leere den Puffer...").
    3. Bereinige jeden persistenten Zustand, der durch das Skript erzeugt wurde, z.B. erzeugte Dateien (falls zutreffend).
    
- Behandele folgende Randfälle korrekt:
    - leere Eingabe
    - große Mengen an Input zwischen den Puffer-Leerungen
    - gleichzeitige Ausführung (mehrere Instanzen des Skripts laufen gleichzeitig)
    
**[1]** Beenden des Skripts kann passieren, wenn:
- die Standardeingabe `stdin` geschlossen wird (es gibt nichts mehr zu lesen).
- das Skript mit den Signalen `INT` oder `TERM` zum Beenden aufgefordert wird (die Behandlung von `KILL` ist unmöglich *by design*).

Bitte beachten Sie die ausführlichen Tests in den folgenden Zellen, mit denen die Funktionalität des Skriptes vollständig geprüft werden kann!

In [ ]:
# ## Aufgabe 1 - Buffers and Traps

# Erstellen Sie ein Skript namens `buffer.sh` im Verzeichnis `$WORK_DIR` (das Arbeitsverzeichnis, das am Anfang dieses Notebooks definiert wurde).
# Stellen Sie sicher, dass das Skript ausführbar ist.
# Sie können es in eine Datei im `$SHEET_DIR` schreiben und von dort ins `$WORK_DIR` kopieren, es mit einem Heredoc erstellen, oder wie auch immer Sie wollen.
# Sie müssen nur sicherstellen, dass das Skript im `$WORK_DIR` vorhanden, korrekt benannt und ausführbar ist.

# Das Skript soll folgenden Anforderungen genügen:

# - Lies (zeilenweise) Text aus der Standardeingabe `stdin` und speichere diesen in einer Art Buffer. Verwenden sie dabei auf **keinen Fall den Befehl mkfifo und named pipes(FIFO)**, sondern nutzen sie einen anderen Weg um Buffer zu schreiben.

# - Leere den Puffer, wenn das Skript ein `USR1`-Signal (`SIGUSR1`) empfängt, d.h. (in dieser Reihenfolge):
#     1. Printe die Zeile `flushing buffer` in der Standardfehlerausgabe `stderr`.
#     2. Printe den gesamten Inhalt des Puffers in die Standardausgabe `stdout`.
#     3. Lösche den Inhalt des Puffers.
    
# - Wenn das Skript beendet wird (s. **[1]**), soll es sich wie folgt verhalten (in dieser Reihenfolge):
#     1. Printe die Zeile `script shutting down` in der Standardfehlerausgabe `stderr`.
#     2. Leere den Puffer ein letztes Mal (siehe obige Definition von "Leere den Puffer...").
#     3. Bereinige jeden persistenten Zustand, der durch das Skript erzeugt wurde, z.B. erzeugte Dateien (falls zutreffend).
    
# - Behandele folgende Randfälle korrekt:
#     - leere Eingabe
#     - große Mengen an Input zwischen den Puffer-Leerungen
#     - gleichzeitige Ausführung (mehrere Instanzen des Skripts laufen gleichzeitig)
    
# **[1]** Beenden des Skripts kann passieren, wenn:
# - die Standardeingabe `stdin` geschlossen wird (es gibt nichts mehr zu lesen).
# - das Skript mit den Signalen `INT` oder `TERM` zum Beenden aufgefordert wird (die Behandlung von `KILL` ist unmöglich *by design*).

# Bitte beachten Sie die ausführlichen Tests in den folgenden Zellen, mit denen die Funktionalität des Skriptes vollständig geprüft werden kann!

cat > buffer.sh <<'EOF'
#!/bin/bash

# Buffer to store input
BUFFER=""

# Function to flush buffer
flush_buffer() {
    echo "flushing buffer" >&2
    echo -n "$BUFFER"
    BUFFER=""
}

# Function to clean up
cleanup() {
    echo "script shutting down" >&2
    flush_buffer
    # Clean up any persistent state
}

# Register cleanup function
trap cleanup EXIT

# Register flush_buffer function for SIGUSR1
trap flush_buffer USR1

# Read from stdin
while read -r line; do
    BUFFER+="$line"$'\n'
done
EOF

chmod +x buffer.sh

In [ ]:
(set -e
cd $WORK_DIR
# check the scripts existence
[[ -e "buffer.sh" ]] || (echo "script not found" && false)
[[ -x "buffer.sh" ]] || (echo "script not executable" && false)
)

In [ ]:
(set -e
cd $WORK_DIR
# run the script in the foreground (without any signal interaction)
[[ $(echo  "ad-hoc run the script" | ./buffer.sh 2> /dev/null) == "ad-hoc run the script" ]] || (echo "ad-hoc output is incorrect" && false)
)

In [ ]:
(set -e
cd $WORK_DIR
# check stderr messages
[[ $(echo "ad-hoc run the script" | ./buffer.sh 2>&1 >/dev/null | grep 'script shutting down') ]] || (echo "shutdown message missing" && false)
[[ $(echo "ad-hoc run the script" | ./buffer.sh 2>&1 >/dev/null | grep 'flushing buffer' | wc -l) -eq 1 ]] || (echo "flushing message missing or printed too often" && false)
)

In [ ]:
(set -e
cd $WORK_DIR
# test multi-flush scenarios
# setup instrumentation
[[ -e "input.txt" ]] && rm "input.txt"; touch "input.txt"
[[ -e "output.txt" ]] && rm "output.txt"; touch "output.txt"
[[ -e "error.txt" ]] && rm "error.txt"; touch "error.txt"
# start the script (in the background)
tail -f input.txt | ./buffer.sh  > output.txt 2>error.txt &
# check that the output is empty in the beginning
sleep 2.1; sync
[[ $(cat output.txt) == "" ]] || (echo "output is not empty in the beginning" && false)
# write some content
echo "some text" >> input.txt
sleep 2.1; sync
[[ $(cat output.txt) == "" ]] || (echo "output is not empty before flushing" && false)
# signal flush
pkill -SIGUSR1 buffer.sh
sleep 2.1; sync
[[ $(grep "some text" output.txt) ]] || (echo "output is not correct after flushing" && false)
# write some more content
echo "more text" >> input.txt
sleep 2.1; sync
[[ -z $(grep "more text" output.txt) ]] || (echo "output is not correct before shutdown" && false)
# shutdown the script
pkill buffer.sh
sleep 2.1; sync
[[ $(grep "more text" output.txt) ]] || (echo "output is not correct after shutdown" && false)
)

In [ ]:
(set -e
cd $WORK_DIR
)

In [ ]:
# cleanup
# stop any leftover jobs (should not be necessary on a correct implementation)
jobs -p | xargs -I {} kill {}
sleep 2
# kill any leftover jobs to make sure
jobs -p | xargs -I {} kill -s SIGKILL {}
rm -rf $WORK_DIR